<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">CONTENT </h1>
<br>

* [Add Libaries](#1)
* [Preproccess](#2)
* [Prepare And Run Model](#3)
* [Visualize Model Results](#4)

<a id="1"> </a>
# Add Libaries


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tensorflow import keras
from keras.models import Sequential,load_model
from keras.layers import Dense,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Dropout,MaxPooling2D,Activation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator as Imgen

from PIL import Image
from sklearn.metrics import confusion_matrix,classification_report

import os

import warnings
warnings.filterwarnings("ignore")

<a id="2"> </a>
# Preproccess


In [ ]:
train_df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
train_path = "../input/plant-pathology-2021-fgvc8/train_images"
test_path = "../input/plant-pathology-2021-fgvc8/test_images"

In [ ]:
train_df.head(3)

In [ ]:
train_df.info()

In [ ]:
labels = train_df["labels"].value_counts().index
data = train_df["labels"].value_counts().values

In [ ]:
data

In [ ]:
explode = (0, 0.1, 0, 0,0,0,0,0,0,0,0,0)
pie, ax = plt.subplots(figsize=[10,6])
plt.pie(x=data, autopct="%.1f%%",explode=explode, labels=labels, pctdistance=0.5)
plt.title("Delivery Tips by type", fontsize=14);
pie.savefig("DeliveryPieChart.png")

In [ ]:
class_num = len(train_df["labels"].value_counts())

In [ ]:
datagen = Imgen(preprocessing_function=keras.applications.xception.preprocess_input,
                 rotation_range=4,
                  shear_range=0.2,
                  zoom_range=0.15,
                  horizontal_flip=True,
                  validation_split=0.2
                 )

In [ ]:
train_ds = datagen.flow_from_dataframe(
    train_df,
    directory = train_path, 
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123
)


val_ds = datagen.flow_from_dataframe(
    train_df,
    directory = train_path,
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123
)

In [ ]:
x,y = next(train_ds)
x.shape

<a id="3"> </a>
# Prepare And Run Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16()

In [ ]:
model = Sequential()
vgg_layers = vgg.layers

for i in range(len(vgg_layers)-1):
    model.add(vgg_layers[i])

In [ ]:
model.summary()

In [ ]:
for l in vgg_layers:
    l.trainable = False

In [ ]:
model.add(Dense(class_num , activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy" , optimizer = "adam" , metrics=["accuracy"])

In [ ]:
batch_size = 32

In [ ]:
hist = model.fit_generator(train_ds,
                           steps_per_epoch = 14900//batch_size,
                           epochs = 15,
                           validation_data = val_ds,
                           validation_steps = 3700//batch_size)

<a id="4"> </a>
# Visualize Model Results

In [ ]:
plt.plot(hist.history["acc"],label = "training acc")
plt.plot(hist.history["val_acc"],label = "validation acc")